In [1]:
!pip install getorg --upgrade
import pandas as pd
import getorg
from geopy import Nominatim

Requirement already up-to-date: getorg in /srv/paws/lib/python3.4/site-packages
Requirement already up-to-date: geopy in /srv/paws/lib/python3.4/site-packages (from getorg)
Requirement already up-to-date: pygithub in /srv/paws/lib/python3.4/site-packages (from getorg)
Requirement already up-to-date: retrying in /srv/paws/lib/python3.4/site-packages (from getorg)
Requirement already up-to-date: six>=1.7.0 in /srv/paws/lib/python3.4/site-packages (from retrying->getorg)
IPywidgets and ipyleaflet support enabled.


In [2]:
articles = pd.read_csv("talks.tsv", sep="\t", header=0)
articles

,date,talk_type,title,venue,institution,geoloc,url,slug,summary,description
0,2016-08-13,Talk,Community Sustainability in Wikipedia: A Revie...,PyData SF,NumFocus,"San Francisco, CA",NaN,pydata-community-sustainability,Wikipedia relies on one of the world’s largest...,Wikipedia relies on one of the world’s largest...
1,2016-07-16,Talk,Governing Open Source Projects at Scale: Lesso...,SciPy,NaN,"Austin, Texas",NaN,scipy-governing-scale,"Many open source, volunteer-driven projects be...","Many open source, volunteer-driven projects be..."
2,2016-06-14,Talk,Administrative Support Bots in Wikipedia: How ...,Communicating with Machines workshop,International Communication Association,"Fukuoka, Japan",NaN,ica-communicating-with-machines,I discuss cases from a multi-year ethnographic...,I discuss cases from a multi-year ethnographic...
3,2016-06-11,Panelist,Drowning in Data: Industry and Academic Approa...,Annual Meeting of the International Communicat...,NaN,"Fukuoka, Japan",NaN,ica-drowning-in-data,This panel extends discusses the potentials an...,"In the past five years, as “big data” research..."
4,2016-06-09,Talk,Successor Systems: Lessons for Big Data From F...,Big Data: Critiques and Alternatives workshop,International Communication Association,"Fukuoka, Japan",NaN,ica-successor-systems,I discuss four data-intensive activist project...,The concept of successor systems extends Hardi...
5,2016-06-08,Talk,"Algorithms as agents of gatekeeping, governanc...","Algorithms, Automation, and Politics workshop",International Communication Association,"Fukuoka, Japan",NaN,ica-algorithms-gatekeeping,I discuss how algorithmic systems are deployed...,This talk is based on a multi-year ethnographi...
6,2016-02-17,Talk,Scraping Wikipedia Data,"The Hacker Within, BIDS",Berkeley Institute for Data Science,"Berkeley, CA",http://www.thehackerwithin.org/berkeley/posts/...,thw-scraping-wikipedia,A tutorial (with Jupyter notebooks) about how ...,A tutorial (with Jupyter notebooks) about how ...
7,2016-01-16,Talk,Why bots are my favorite contribution to Wikip...,Wikipedia 15th Anniversary Birthday Bash,Wikimedia Foundation,"San Francisco, CA",NaN,wiki15-bots,A short talk to open up an event celebrating t...,A short talk to open up an event celebrating t...
8,2015-11-12,Talk,The Bot Multiple: Unpacking the Materialities ...,Annual Meeting of the Society for the Social S...,NaN,"Denver, CO",NaN,4s-bot-multiple,I examine the roles that automated software ag...,This paper examines the roles that automated s...
9,2015-11-06,Panelist,Crowdsourcing: Theoretical Considerations,Crowdsourcing and the Academy Symposium,UC-Berkeley,"Berkeley, CA",http://hssa.berkeley.edu/crowdsourcing-symposium,crowdsourcing-academy,A panel discussing how academics use crowdsour...,NaN


In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;",
    ">": "&gt;",
    "<": "&lt;",
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [4]:
geocoder = Nominatim()

In [5]:
import os
loc_dict = {}
count = 0
for row, item in articles.iterrows():
    md_filename = str(item.date) + "-" + item.slug + ".md"
    html_filename = str(item.date) + "-" + item.slug 
    year = item.date[:4]

    map_text = item.date + ", " + item.talk_type + " at " + item.venue + "<br/>\n"
    map_text += "<a href='https://staeiou.github.io/talks/" + html_filename + "' target='_blank'>"
    map_text += html_escape(item.title) + "</a>"
    loc = geocoder.geocode(item.geoloc)
    loc_dict[map_text] = loc
    count += 1
    
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    if item.talk_type is not None:
        md += 'talk_type: "' + item.talk_type + '"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if item.venue is not None:
        md += 'venue: "' + item.venue + '"\n'
        
    if item.date is not None:
        md += "date: " + str(item.date) + "\n"
    
    if item.geoloc is not None:
        md += 'location: "' + str(item.geoloc) + '"\n'
       
    
    if len(str(item.summary))>10:
        md += 'excerpt: "'
        md += html_escape(item.summary) + '"\n'
    elif len(str(item.description))>10:
        if len(str(item.description))>200:
            md += 'excerpt: "'
            md += html_escape(item.description[:200])
            md += '..."\n'
        else:
            md += 'excerpt: "'
            md += html_escape(item.description) + '"\n'
    
    md += "---\n"
    
    if isinstance(item.url, str):
        md += "\n<a href='" + str(item.url) + "'>Link to more information</a>\n" 
    
    if len(str(item.description))>10:
        md += "\n" + html_escape(item.description) + "\n"
    md_filename = os.path.basename(md_filename)
    print(md)
    
    with open(md_filename, 'w') as f:
        f.write(md)

---
title: "Community Sustainability in Wikipedia: A Review of Research and Initiatives"
collection: talks
talk_type: "Talk"
permalink: /talks/2016-08-13-pydata-community-sustainability
venue: "PyData SF"
date: 2016-08-13
location: "San Francisco, CA"
excerpt: "Wikipedia relies on one of the world’s largest open collaboration communities. Since 2001, the community has grown substantially and faced many challenges. This presentation reviews research and initiatives around community sustainability in Wikipedia that are relevant for many open source projects, including issues of newcomer retention, governance, automated moderation, and marginalized groups."
---

Wikipedia relies on one of the world’s largest open collaboration communities. Since 2001, the community has grown substantially and faced many challenges. This presentation reviews research and initiatives around community sustainability in Wikipedia that are relevant for many open source projects, including issues of newcomer re

In [6]:
m = getorg.orgmap.create_map_obj()
getorg.orgmap.map_location_dict(m, loc_dict)

In [7]:
getorg.orgmap.output_html_cluster_map(loc_dict, hashed_usernames=False)

'Written map to cluster_map/'